In [1]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

#engine = create_engine("sqlite:///:memory:")
engine = create_engine(
    'snowflake://{user}:@{account_identifier}/{database}/{schema_name}?authenticator={authenticator}&warehouse={warehouse}'.format(
        user='japneet.singh@lacework.net',
        account_identifier='LWDEV',
        database='japneet_test_db',
        schema_name='PUBLIC',
        authenticator='externalbrowser',
        warehouse='DEV_TEST',
    )
)
metadata_obj = MetaData(bind=engine)

C:\Users\Japneet\AppData\Local\Temp\ipykernel_22832\2997248023.py:14: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  metadata_obj = MetaData(bind=engine)


In [2]:
# create city SQL table
#table_name = "city_stats"
#city_stats_table = Table(
#    table_name,
#    metadata_obj,
#    Column("city_name", String(16), primary_key=True),
#    Column("population", Integer),
#    Column("country", String(16), nullable=False),
#)
#metadata_obj.create_all()

In [3]:
#from sqlalchemy import insert
#rows = [
#    {"city_name": "Toronto", "population": 2731571, "country": "Canada"},
#    {"city_name": "Tokyo", "population": 13929286, "country": "Japan"},
#    {"city_name": "Berlin", "population": 600000, "country": "United States"},
#]
#for row in rows:
#    stmt = insert(city_stats_table).values(**row)
#    with engine.connect() as connection:
#        cursor = connection.execute(stmt)

In [4]:
#from llama_index import SQLDatabase

#sql_database = SQLDatabase(engine, include_tables=["city_stats"])


In [5]:
#from llama_index import GPTSQLStructStoreIndex, SQLDatabase
#index = GPTSQLStructStoreIndex(
#    [],
#    sql_database=sql_database, 
#    table_name="city_stats",
#)


In [6]:
#from llama_index import download_loader

#WikipediaReader = download_loader("WikipediaReader")
#wiki_docs = WikipediaReader().load_data(pages=['Toronto', 'Berlin', 'Tokyo'])


In [7]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [9]:
#from llama_index import GPTSQLStructStoreIndex, SQLDatabase

#sql_database = SQLDatabase(engine, include_tables=["city_stats"])
# NOTE: the table_name specified here is the table that you
# want to extract into from unstructured documents.
#index = GPTSQLStructStoreIndex(
#    wiki_docs, 
#    sql_database=sql_database, 
#    table_name="city_stats",
#)

In [10]:
index = GPTSQLStructStoreIndex(
    [],
    sql_database=sql_database, 
    table_name="city_stats",
)

C:\Users\Japneet\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 0 tokens
INFO:snowflake.connector.cursor:query: [select current_database(), current_schema();]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SHOW /* sqlalchemy:_get_schema_primary_keys */PRIMARY KEYS IN SCHEMA japneet_tes...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT /* sqlalchemy:_get_schema_columns */ ic.table_name, ic.column_name, ic.da...]
INFO:

In [11]:
# set Logging to DEBUG for more detailed outputs
response = index.query("Which city has the highest population? Also show the population.", mode="default")
print(response)

INFO:root:> Table desc str: Schema of table city_stats:
Table 'city_stats' has columns: city_name (VARCHAR(16)), population (DECIMAL(38, 0)), country (VARCHAR(16)).

INFO:snowflake.connector.cursor:query: [SELECT city_name, population FROM city_stats ORDER BY population DESC LIMIT 1;]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [ROLLBACK]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:root:> [query] Total LLM token usage: 166 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


[('Tokyo', 13929286)]


In [12]:
response.extra_info['sql_query']

'SELECT city_name, population \nFROM city_stats \nORDER BY population DESC \nLIMIT 1;'

In [18]:
# set Logging to DEBUG for more detailed outputs
response = index.query("What is the country of the city has the highest population?", mode="default")
print(response)

INFO:root:> Table desc str: Schema of table city_stats:
Table 'city_stats' has columns: city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16)).

INFO:root:> [query] Total LLM token usage: 158 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


[('Japan',)]


In [19]:
response.extra_info['sql_query']

'SELECT country \nFROM city_stats \nORDER BY population DESC \nLIMIT 1;'

In [13]:
# manually set text
#city_stats_text = (
#    "This table gives information regarding the population and country of a given city.\n"
#    "The user will query with codewords, where 'foo' corresponds to population and 'bar'"
#    "corresponds to city."
#)
#table_context_dict={"city_stats": city_stats_text}
#context_builder = SQLContextContainerBuilder(sql_database, context_dict=table_context_dict)
#context_container = context_builder.build_context_container()

# building the index
#index = GPTSQLStructStoreIndex(
#    wiki_docs, 
#    sql_database=sql_database, 
#    table_name="city_stats",
#    sql_context_container=context_container
#)

In [26]:
from llama_index import GPTSQLStructStoreIndex, SQLDatabase, GPTSimpleVectorIndex
from llama_index.indices.struct_store import SQLContextContainerBuilder

sql_database = SQLDatabase(engine)
# build a vector index from the table schema information
context_builder = SQLContextContainerBuilder(sql_database)
table_schema_index = context_builder.derive_index_from_context(
    GPTSimpleVectorIndex
)

query_str = "Which city has the highest population?"

# query the table schema index using the helper method
# to retrieve table context
context_builder.query_index_for_context(
    index=table_schema_index,
    query_str=query_str,
    store_context_str=True
)
context_container = context_builder.build_context_container()

# query the SQL index with the table context
response = index.query(query_str, sql_context_container=context_container)
print(response)

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 52 tokens
INFO:root:> [query] Total LLM token usage: 135 tokens
INFO:root:> [query] Total embedding token usage: 23 tokens
INFO:root:> Table desc str: 
Table 'city_stats':
city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16))
INFO:root:> [query] Total LLM token usage: 134 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


[('Tokyo',)]
